In [1]:
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import numpy as np
import re
import pandas as pd
import gensim
from collections import namedtuple
from keras import models
from keras import layers

    100% |████████████████████████████████| 23.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 14.6MB/s 
    100% |████████████████████████████████| 133kB 16.5MB/s 
    100% |████████████████████████████████| 61kB 22.8MB/s 
    100% |████████████████████████████████| 4.7MB 5.1MB/s 
    100% |████████████████████████████████| 552kB 26.1MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [2]:
import io
import requests
url1="https://drive.google.com/uc?export=download&id=1TjmolGwsUQK0OQ143E_gkksZ6CQU9_aK"
url2="https://drive.google.com/uc?export=download&id=1WcvgkDWyL4vY9jfvh5bm1zSpCNFxyOOW"
print("Starting Download")
s1=requests.get(url1).content
s2=requests.get(url2).content
fileTRAIN=pd.read_csv(io.StringIO(s2.decode('utf-8')))
fileTEST=pd.read_csv(io.StringIO(s1.decode('utf-8')))
print("Finished")

Starting Download
Finished


In [10]:
fileTRAIN.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
d2vmodeltextdata = fileTRAIN['text'].values

In [0]:
appenddata = fileTEST['text'].values
alltextdata = np.concatenate((d2vmodeltextdata,appenddata))

In [16]:
print(alltextdata)

['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing ema

In [29]:
alltextdatalist = alltextdata.tolist()

str

In [0]:
#now used saved model here dont rerun all the above shit

In [34]:
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(alltextdata)]
max_epochs = 20
vec_size = 100
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  from ipykernel import kernelapp as app


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [10]:
d2vmodeltitledata = fileTRAIN['title'].values
appendtitledata = fileTEST['title'].values
alltitledata = np.concatenate((d2vmodeltitledata,appendtitledata))
alltitledatalist = alltitledata.tolist()
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(alltitledata)]
max_epochs = 20
vec_size = 25
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2vfortitle25dim.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [21]:
d2vmodelauthordata = fileTRAIN['author'].values
appendauthordata = fileTEST['author'].values
alltitledata = np.concatenate((d2vmodelauthordata,appendauthordata))
tagged_data = [TaggedDocument(words=word_tokenize(str(_d).lower()), tags=[str(i)]) for i, _d in enumerate(alltitledata)]
max_epochs = 20
vec_size = 25
alpha = 0.025
model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save("d2vforauthor25dim.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
Model Saved


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [5]:
!ls 'drive/My Drive/NewsDataset'

d2v_article_models_trained	  d2v.model.wv.vectors.npy
d2vforauthor25dim.model		  d2v_title_model_trained
d2vfortitle25dim.model		  News Dataset.zip
d2vfortitle.model		  News Dataset.zip (Unzipped Files)
d2v.model			  VectorizedDataframe.csv
d2v.model.trainables.syn1neg.npy  VectorizedDataframeTest.csv


In [0]:
modelarticle = Doc2Vec.load('drive/My Drive/NewsDataset/d2v.model')
modeltitle = Doc2Vec.load('drive/My Drive/NewsDataset/d2vfortitle25dim.model')
modelauthor = Doc2Vec.load('drive/My Drive/NewsDataset/d2vforauthor25dim.model')

In [0]:
def vectorConvertorArticle(obj):
  obj = str(obj).lower()
  data = word_tokenize(obj)
  return modelarticle.infer_vector(data)

def vectorConvertorTitle(obj):
  obj = str(obj).lower()
  data = word_tokenize(obj)
  return modeltitle.infer_vector(data)

def vectorConvertorAuthor(obj):
  obj = str(obj).lower()
  data = word_tokenize(obj)
  return modelauthor.infer_vector(data)

In [0]:
fileTRAIN['article_vector'] = fileTRAIN['text'].apply(vectorConvertorArticle)
fileTRAIN['title_vector'] = fileTRAIN['title'].apply(vectorConvertorTitle)
fileTRAIN['author_vector'] = fileTRAIN['author'].apply(vectorConvertorAuthor)

In [0]:
fileTRAIN.to_csv('VectorizedDataframe.csv')

In [0]:
fileTEST['article_vector'] = fileTEST['text'].apply(vectorConvertorArticle)
fileTEST['title_vector'] = fileTEST['title'].apply(vectorConvertorTitle)
fileTEST['author_vector'] = fileTEST['author'].apply(vectorConvertorAuthor)


In [0]:
fileTEST.to_csv('VectorizedDataframeTest.csv')

In [8]:
fileTRAIN.head()

,id,title,author,text,label,article_vector,title_vector,author_vector
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[1.6827588, -0.46717584, 1.1852177, 0.82204306...","[0.0823192, -0.26001894, 0.117337584, 0.270736...","[-0.030947596, -0.014029763, 0.0037813466, 0.0..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"[1.0740684, 1.6764545, 0.99277586, 1.2082169, ...","[0.046257842, -0.27694014, -0.1521388, 0.04457...","[-0.010836543, -0.019938458, -0.0074933944, 0...."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,"[0.26666343, 0.8204189, -1.8018087, 1.9324995,...","[0.20127644, -0.58856887, -0.026112111, 0.4397...","[-0.01088245, 0.061993875, 0.1172995, -0.02802..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,"[0.7810626, 0.94763523, -0.32702693, 1.8603771...","[0.3121563, -0.47003347, -0.001772513, 0.29951...","[-0.014858428, -0.022646226, 0.07036754, 0.056..."
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,"[0.2852846, -0.41440225, 0.33196193, 0.7258399...","[0.09513792, -0.42330456, 0.3411817, 0.0998370...","[-0.012663558, 0.0027816598, -0.0040997434, -0..."


In [0]:
def parenthesesCleaner(line):
  
  matching = re.match( r'.*([[])(.*)([]])', str(line), re.M|re.I)
  if matching:
    ret = matching.group(2)
    return ret
  else:
    print("Error")
  

In [0]:
x = fileTRAIN['article_vector'].values
y = fileTRAIN['title_vector'].values
z = fileTRAIN['author_vector'].values
xy=np.empty_like(x)

In [0]:
for i in range(len(x)):
  xy[i] = np.append(x[i],y[i])
yz=np.empty_like(xy)


In [0]:
for i in range(len(xy)):
  yz[i] = np.append(xy[i],z[i])

In [0]:
vectors = pd.DataFrame(yz,columns=['features'])

In [0]:
vectors['label'] = fileTRAIN['label']

In [0]:
features = vectors['features'].values
labels = vectors['label'].values

In [59]:
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size= 0.25, random_state=27)


array([0, 0, 1, ..., 0, 0, 0])

In [0]:
y_train=y_train.tolist()

list

In [0]:
x1 = fileTEST['article_vector'].values
y1 = fileTEST['title_vector'].values
z1 = fileTEST['author_vector'].values
xy1=np.empty_like(x1)
for i in range(len(x1)):
  xy1[i] = np.append(x1[i],y1[i])
yz1=np.empty_like(xy1)
for i in range(len(xy1)):
  yz1[i] = np.append(xy1[i],z1[i])
vectors_test = pd.DataFrame(yz1,columns=['features'])
features_test = vectors['features'].values
labels_test = vectors['label'].values

In [74]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features,labels, test_size= 0.25, random_state=27)
x_train=x_train.tolist()
y_train=y_train.tolist()
x_test=x_test.tolist()
y_test=y_test.tolist()
classifier = MLPClassifier(hidden_layer_sizes=(150,75,100), max_iter=500, alpha=0.0001,
                     solver='lbfgs', verbose=10,  random_state=21, tol=0.000000001)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
accuracy_score(y_test, y_pred)

0.958076923076923